In [1]:

import paddle
import paddleslim
from paddle.quantization import QuantConfig
from paddle.quantization.quanters import FakeQuanterWithAbsMaxObserver
from paddle.quantization.quanters.abs_max import (
    FakeQuanterWithAbsMaxObserverLayer,
)
from paddle.vision.models import resnet18
from paddleslim.quant import SlimQAT
from paddleslim.quant.constraints import FreezedConvBNConstraint
model = resnet18()

/root/miniconda3/lib/python3.8/site-packages/paddle/fluid/framework.py:507: UserWarning: PaddlePaddle version 2.3.0 or higher is required, but 0.0.0 installed, Maybe you are using a develop version, please make sure the version is good with your code.


In [2]:
quanter = FakeQuanterWithAbsMaxObserver(moving_rate=0.9)
q_config = QuantConfig(activation=quanter, weight=quanter)
q_config.add_constraints(FreezedConvBNConstraint())
qat = SlimQAT(q_config)
x = paddle.rand([1, 3, 224, 224])
quant_model = qat.quantize(model, inplace=True, inputs=x)
#paddle.jit.save(quant_model, "./qat_model", input_spec=[x])
# convert_model = qat.convert(quant_model)
# paddle.jit.save(convert_model, "./fp_infer", input_spec=[x])
# out = quant_model(x)
# out.backward()

/root/miniconda3/lib/python3.8/site-packages/paddle/nn/layer/norm.py:771: UserWarning: When training, we now always track global mean and variance.


In [ ]:
print(quant_model)

### 导出ONNX格式模型

In [ ]:
!paddle2onnx --model_dir ./ \
            --model_filename fp_infer.pdmodel \
            --params_filename fp_infer.pdiparams \
            --save_file fp_infer.onnx \
            --opset_version 13

In [ ]:
import paddleslim
print(dir(paddleslim.quant))
print(paddleslim.__file__)